In [7]:
'''
Author: Aditya Jain
Date  : 3rd August, 2021
About : Annotation generation using CV methods for DL-based localization task
'''
import cv2
import os
import json

SCALE_PER       = 0.7    # scaling of the image
NOISE           = 15     # noise factor for removal
AREA_THRESHOLD  = 0.05   # in percentage

DATA_DIR        = 'train/'
SAVE_DIR        = 'dl_train/'

In [23]:
def resize_custom(image, scale):
    '''
    returns the downscaled image
    '''
    width     = int(image.shape[1]*scale)
    height    = int(image.shape[0]*scale)
    dim       = (width, height)
    
    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

def bbox_label_data(input_image, bg_image, noise_fac, area_thresh):
    '''returns the bbox coordinates and labels for multiple blob/moth in image'''

    # conversion to grayscale
    input_gray   = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    bg_gray      = cv2.cvtColor(bg_image, cv2.COLOR_BGR2GRAY)
    diff         = cv2.absdiff(input_gray,bg_gray)
    ret, mask    = cv2.threshold(diff, noise_fac, 255, cv2.THRESH_BINARY)
    
    # calculates connected components and relevant stats
    output       = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
    stats        = output[2]
    
    # calculates the bounding boxes
    total_pix    = mask.shape[0]*mask.shape[1]  # calculates the total number of pixels
    total_rows   = stats.shape[0]
    bbox_list    = []
    label_list   = []
    for i in range(1, total_rows):
        item = stats[i,:]
        area = (item[-1]/total_pix)*100
        if area>=area_thresh:
            bbox_list.append([int(item[0]), int(item[1]), int(item[0]+item[2]), int(item[1]+item[3])])
            label_list.append(1)     # there is only one label, i.e., insect
    
    return bbox_list, label_list

In [24]:
bg_img      = cv2.imread('bg_imageseq1.jpg')
bg_resized  = resize_custom(bg_img, SCALE_PER)
annot_data  = {}

images      = os.listdir(DATA_DIR)

for image in images:
    if image.endswith('.jpg'):
        img_path       = DATA_DIR + image
        inp_img        = cv2.imread(img_path)
        inp_resized    = resize_custom(inp_img, SCALE_PER)    
        b_data, l_data = bbox_label_data(inp_resized, bg_resized, NOISE, AREA_THRESHOLD)
    
        if len(l_data):
            cv2.imwrite(SAVE_DIR+image, inp_resized)     # saving original resized image        
            annot_data[image] = [b_data, l_data]
        
with open('annotation_data.json', 'w') as outfile:
    json.dump(annot_data, outfile)

#### CV annotation for test data for comparison later

In [26]:
def bbox_testdata(input_image, bg_image, noise_fac, area_thresh):
    '''draws cv-based bbox annotation for test/crowded images'''

    # conversion to grayscale
    input_gray   = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    bg_gray      = cv2.cvtColor(bg_image, cv2.COLOR_BGR2GRAY)
    diff         = cv2.absdiff(input_gray,bg_gray)
    ret, mask    = cv2.threshold(diff, noise_fac, 255, cv2.THRESH_BINARY)
    
    # calculates connected components and relevant stats
    output       = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)
    stats        = output[2]
    
    # calculates the bounding boxes
    total_pix    = mask.shape[0]*mask.shape[1]  # calculates the total number of pixels
    total_rows   = stats.shape[0]

    for i in range(1, total_rows):
        item = stats[i,:]
        area = (item[-1]/total_pix)*100
        if area>=area_thresh:
            cv2.rectangle(input_image,(item[0], item[1]),(item[0]+item[2], item[1]+item[3]),(0,0,255),3)            
                
    return input_image

In [28]:
TEST_DATA_DIR    = 'test/'
SAVE_INP_DIR     = 'dl_test/orig_resized_images/'
SAVE_CVANNOT_DIR = 'dl_test/cv_bbox/'

bg_img           = cv2.imread('bg_imageseq1.jpg')
bg_resized       = resize_custom(bg_img, SCALE_PER)

images           = os.listdir(TEST_DATA_DIR)

for image in images:
    if image.endswith('.jpg'):
        img_path       = TEST_DATA_DIR + image
        inp_img        = cv2.imread(img_path)
        inp_resized    = resize_custom(inp_img, SCALE_PER)
        cv2.imwrite(SAVE_INP_DIR+image, inp_resized)
        
        ann_image      = bbox_testdata(inp_resized, bg_resized, NOISE, AREA_THRESHOLD)
        cv2.imwrite(SAVE_CVANNOT_DIR+image, ann_image)
